<a href="https://colab.research.google.com/github/taremwatadeo/Artificial-Intelligence/blob/master/CAT2_sentiment_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing necessary libraries
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
import torch

In [ ]:
!pip install huggingface_hub[hf_xet]

In [ ]:
# loading the amazon polarity dataset
# This dataset contains Amazon product reviews labeled as positive (1) or negative (0)
dataset = load_dataset("amazon_polarity")

In [ ]:
# Spliting the dataset into train and validation sets
train_val = dataset["train"].train_test_split(test_size=0.1)
train_dataset = train_val["train"].shuffle(seed=42).select(range(1000))
eval_dataset = train_val["test"].shuffle(seed=42).select(range(200))

In [ ]:
# Loading the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# Defining a function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True, max_length=128)

In [ ]:
# Apply tokenization to the train and validation datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Preparing the datasets for the Trainer
# here we remove unnecessary columns and rename 'label' to 'labels' (required by Trainer)
tokenized_train = tokenized_train.remove_columns(["content"])
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_train.set_format("torch")  # we Convert to PyTorch tensors

In [ ]:
tokenized_eval = tokenized_eval.remove_columns(["content"])
tokenized_eval = tokenized_eval.rename_column("label", "labels")
tokenized_eval.set_format("torch")

In [ ]:
# we load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip show transformers

Name: transformers
Version: 4.51.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
from transformers import TrainingArguments


In [ ]:
# we define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

In [ ]:
import wandb

In [ ]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
wandb.init()

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=189, training_loss=0.0957789244475188, metrics={'train_runtime': 1881.0588, 'train_samples_per_second': 1.595, 'train_steps_per_second': 0.1, 'total_flos': 99350548992000.0, 'train_loss': 0.0957789244475188, 'epoch': 3.0})

In [ ]:
model.save_pretrained("fine_tuned_distilbert_sentiment")
tokenizer.save_pretrained("fine_tuned_distilbert_sentiment")

('fine_tuned_distilbert_sentiment/tokenizer_config.json',
 'fine_tuned_distilbert_sentiment/special_tokens_map.json',
 'fine_tuned_distilbert_sentiment/vocab.txt',
 'fine_tuned_distilbert_sentiment/added_tokens.json')

In [ ]:
def predict_sentiment(review):
    # Tokenizing the input review
    inputs = tokenizer(review, padding=True, truncation=True, max_length=128, return_tensors="pt")
    # Move inputs to the same device as the model (GPU if available)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Positive" if prediction == 1 else "Negative"

In [ ]:
print("Testing with predefined reviews:")
reviews = [
    "This product is amazing! I love it.",
    "Terrible product, it broke after one use.",
    "It's okay, nothing special."
]

Testing with predefined reviews:


In [ ]:
for review in reviews:
    print(f"Review: {review}")
    print(f"Sentiment: {predict_sentiment(review)}")
    print()

Review: This product is amazing! I love it.
Sentiment: Positive

Review: Terrible product, it broke after one use.
Sentiment: Negative

Review: It's okay, nothing special.
Sentiment: Negative



In [ ]:
user_review = input("Enter your own review: ")
print(f"Sentiment: {predict_sentiment(user_review)}")

Enter your own review: nothing special
Sentiment: Negative
